In [1]:
import sys
print(sys.executable)
print(sys.version)

C:\Users\mrkcb\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe
3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]


In [2]:
import pandas as pd
print(pd.__version__)


2.3.3


In [1]:
from bs4 import BeautifulSoup
import pandas as pd 
import os

def get_name_price(html):
    data = []

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    tables = soup.find_all('div', class_='pt-2')

    relevant = tables[0]

    listitems = relevant.find_all('li')

    for li in listitems:

        seller_names = li.find_all('span', attrs={'data-qa': 'seller-name'})
        seller_price = li.find_all('span', attrs={'data-qa': 'seller-price'})

        if seller_names and seller_price:
            data.append([seller_names[0].text, seller_price[0].text])

    data = pd.DataFrame(data)
    data.columns = ['pharmacy', 'price']
    return data




ModuleNotFoundError: No module named 'pandas'

In [ ]:
fnames = [fname for fname in os.listdir('.') if fname.endswith('GoodRx.html')]

results = []

for fname in fnames:
    with open(fname, 'r') as f:
        html_content = f.read()
    data = get_name_price(html_content)
    drug_name = fname.split(' ')[0]
    data['name'] = drug_name
    results.append(data)

all_data = pd.concat(results)

all_data.to_csv('all_data.csv', index=False)